In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
file = 'D:\work\Jira_analyses\out_csv.csv'

In [3]:
#load source file
users = pd.read_csv(file,
                        encoding="utf-8-sig",
                       sep=";",
                       usecols =['KEY','AUTHOR_NAME','DATE_CREATED','OLD_VALUE','NEW_VALUE'])

In [4]:
#adding end_dates column to source data frame. end dates are nothing but start_dates shifted on 1 value 
# since each new line has start date which is end date for previous status
end_dates = users['DATE_CREATED'].tolist();
end_dates.pop(0)
end_dates.append('null')
users = users.assign(END_DATES = pd.Series(end_dates))
#print(users.groupby('KEY').tail(2))

In [5]:
#set last end_date for each story as a last update date of the story
oldvalue = users.iloc[0]
for index, row in users.iterrows():
    if oldvalue.KEY != row.KEY :
        users.loc[users.index[index-1],'END_DATES'] = users.loc[users.index[index-1],'DATE_CREATED']  
    oldvalue=row
users.loc[users.index[len(users)-1],'END_DATES'] = users.loc[users.index[len(users)-1],'DATE_CREATED']    
print(users)

             KEY  AUTHOR_NAME         DATE_CREATED               OLD_VALUE  \
0    RMSODS-4179         pr10  31.10.2017 10:13:32           To Be Refined   
1    RMSODS-4179         pr10   15.11.2017 8:57:19               In Refine   
2    RMSODS-4179         IC16   16.11.2017 9:26:13         Ready for Build   
3    RMSODS-4179         IC16    23.11.2017 8:38:0                In Build   
4    RMSODS-4179         IC16   23.11.2017 9:20:37                 In Test   
5    RMSODS-4179         IC16   24.11.2017 9:21:27    Ready for Deployment   
6    RMSODS-4072         IC16     12.9.2017 13:8:9  Sub-task To Be Refined   
7    RMSODS-4072         FC82     2.10.2017 1:8:21         Ready for Build   
8    RMSODS-4072         FC82    5.10.2017 8:31:24    Ready for Deployment   
9    RMSODS-4072         FC82     5.10.2017 8:34:2  Deployed, Not Complete   
10   RMSODS-4064         FC82  25.12.2017 15:45:48  Sub-task To Be Refined   
11   RMSODS-4064         FC82  25.12.2017 15:47:44              

In [11]:
time_diff = users['END_DATES']
time = users[['DATE_CREATED','END_DATES']].apply(lambda x: datetime.strptime(x['END_DATES'],'%d.%m.%Y %H:%M:%S') - datetime.strptime(x['DATE_CREATED'],'%d.%m.%Y %H:%M:%S'), axis=1)
time = []
print()
#datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%d.%m.%Y %H:%M:%S')

0      14 days 22:43:47
1       1 days 00:28:54
2       6 days 23:11:47
3       0 days 00:42:37
4       1 days 00:00:50
5       0 days 00:00:00
6      19 days 12:00:12
7       3 days 07:23:03
8       0 days 00:02:38
9       0 days 00:00:00
10      0 days 00:01:56
11      0 days 00:10:40
12      2 days 16:51:55
13     13 days 00:24:27
14      0 days 00:00:10
15      0 days 00:00:00
16      2 days 19:08:08
17      3 days 04:50:43
18      0 days 00:00:00
19      0 days 16:56:25
20      0 days 00:00:00
21      0 days 00:00:00
22      1 days 16:40:55
23      4 days 22:34:45
24     28 days 10:02:38
25      0 days 00:00:00
26      0 days 23:43:40
27      0 days 03:41:18
28      0 days 00:00:00
29      0 days 19:43:13
             ...       
884     6 days 23:30:30
885   670 days 05:59:22
886     0 days 00:00:00
887     3 days 18:04:31
888     0 days 01:54:21
889     3 days 00:31:28
890     0 days 00:00:00
891     0 days 01:38:19
892     6 days 02:05:54
893    13 days 18:33:31
894     7 days 2

In [7]:
grouped_list = users.groupby('KEY').size()

In [8]:
test = grouped_list.sort_values('KEY')

ValueError: No axis named KEY for object type <class 'pandas.core.series.Series'>

In [ ]:
print(test)